In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

In [ ]:
from decouple import config

In [ ]:
import joblib

In [ ]:
playlist_DF = pd.read_csv('https://raw.githubusercontent.com/ruchitc/music-recommender-system/e16679907663fd3cee6e200cf5b20875717c182a/dataset/SpotifyFeatures.csv')

In [ ]:
playlistDF_num = playlist_DF[['popularity', 'acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness',	'liveness',	'loudness',	'speechiness',	'tempo',	'valence']]
playlistDF_text = playlist_DF[['genre', 'key', 'mode']]

In [ ]:
scaler = MinMaxScaler()
num_df = pd.DataFrame(scaler.fit_transform(playlistDF_num), columns=playlistDF_num.columns)
num_df.reset_index(drop = True, inplace = True)

In [ ]:
text_df = pd.get_dummies(playlistDF_text, prefix=['genre', 'key', 'mode'])
text_df.reset_index(drop = True, inplace = True)    

In [ ]:
final_df = pd.concat([num_df, text_df], axis=1)
final_df['track_id'] = playlist_DF['track_id'].values

In [ ]:
client_id = config("client_id")
client_secret = config("client_secret")

In [ ]:
scope = 'user-library-read'

In [ ]:
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [ ]:
playlist_URL = "https://open.spotify.com/playlist/37i9dQZF1DXbTxeAdrVG2l?si=233815c496e54e41"
playlist_URI = playlist_URL.split("/")[-1].split("?")[0]

In [ ]:
playlist = pd.DataFrame()


for ix, i in enumerate(sp.playlist(playlist_URI)['tracks']['items']):
  #print(i['track']['artists'][0]['name'])
  playlist.loc[ix, 'artist'] = i['track']['artists'][0]['name']
  playlist.loc[ix, 'name'] = i['track']['name']
  playlist.loc[ix, 'id'] = i['track']['id'] # ['uri'].split(':')[2]
  playlist.loc[ix, 'url'] = i['track']['album']['images'][1]['url']
  playlist.loc[ix, 'date_added'] = i['added_at']

playlist['date_added'] = pd.to_datetime(playlist['date_added'])  
    
playlist = playlist[playlist['id'].isin(playlist_DF['track_id'].values)].sort_values('date_added',ascending = False)

In [ ]:
complete_playlist = final_df[final_df['track_id'].isin(playlist['id'].values)]
complete_not_in_playlist = final_df[~final_df['track_id'].isin(playlist['id'].values)]
playlist_final = complete_playlist.drop(columns = "track_id")
complete_playlist_summary_vector = playlist_final.sum(axis = 0)

In [ ]:
non_playlist_df = playlist_DF[playlist_DF['track_id'].isin(complete_not_in_playlist['track_id'].values)]
non_playlist_df['cosine_sim'] = cosine_similarity(complete_not_in_playlist.drop('track_id', axis = 1).values, complete_playlist_summary_vector.values.reshape(1, -1))[:,0]
non_playlist_df_top_20 = non_playlist_df.sort_values('cosine_sim',ascending = False).head(20)